In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from playground import DataGenerator

datasets = DataGenerator()

fig = plt.figure('Available Datasets')
for i, name in enumerate(['circle', 'xor', 'gauss', 'spiral']):
    data, labels = getattr(datasets, name)(200)
    axis = plt.subplot(221+i)
    axis.set_title(name)
    axis.scatter(*zip(*data), c=labels, cmap='bwr')
fig.canvas.draw()

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.optimizers import SGD

# Choose dataset here.
# datasets.circle(), datasets.xor(), datasets.gauss(), datasets.spiral()
data, labels = datasets.gauss()

model = Sequential([
    Dense(4, input_dim=2),
    Activation('relu'),
    Dense(1),
    Activation('relu'),
])

print(model.summary())

sgd = SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
model.compile(optimizer=sgd, loss='mean_squared_error', metrics=['accuracy'])

model.fit(
    data, 
    labels,
    batch_size=10,
    nb_epoch=1000, 
    verbose=0, 
    shuffle=True, 
    validation_split=0
)

fig = plt.figure('Predictions')
plt.scatter(*zip(*data), c=model.predict(data), cmap='bwr')
fig.canvas.draw()